<a href="https://colab.research.google.com/github/majorquev/Introduccion_a_ciencia_de_datos_R_v2/blob/main/Clases/Clase6/Clase6_1_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 5: Clustering


**Autor:** Miguel Jorquera Viguera<br>
**E-mail:** majorquev@gmail.com<br>
**Ayudante:** Rodrigo Morales Méndez<br>
**E-mail ayudante:** rumorale1@gmail.com



Las los algoritmos de clustering corresponden a métodos no supervisados que buscan identificar estructuras de grupos dentro de un conjunto de datos. Dentro de los métodos más comunes, destacan aquellos basados en similitud, donde obervaciones que pertenecen a un mismo cluster son similares entre sí (grupos homogéneos), pero distintas a las observaciones de los otro clusters. 

Dentro de los distintos métodos destacan:


  * Enfoque de particiones 
    * K-means, k-medioids, CLARANS 
  * Enfoque jerárquico
    * Diana, Agnes, BIRCH, CAMELEON
  * Enfoque basado en densidad
    * DBSCAN, OPTICS, DenClue

A continuación se muestra la implementación de algunos de estos algoritmosen R, para mayor información refiérsa a la documentación de la clase y a las referencias citadas.




## Algoritmo k-means



In [ ]:

# K-Means
# Se define el nombre de la base de datos IRIS borrando la variable en donde 
# figura el verdadero nombre de la planta definida por cada fila

data(iris)
iris

dplyr::count(iris,Species)


In [ ]:
base <- iris
base$Species <- NULL
head(base)

In [ ]:
# Se genera la clusterización con k medias con la siguiente estructura: 
# kmeans(datos, numero de clusters que se quiere obtener) # se recomienda nstart > 1

kmeans.resultado <- kmeans(base, centers = 3, nstart = 10 )



Hasta aquí llegó el algoritmo, pero algunas preguntas de interés que buscamos abordar a continuación.

* ¿Cómo podemos visualizar la clusterización realizada?
* ¿Que más podemos hacer además identificar los grupos generados? 
* ¿porqué escogimos 3 grupos? y ¿Cómo podemos  determinar el número óptimo de clusters?

In [ ]:
str(kmeans.resultado)
kmeans.resultado$cluster

base$cl_kmens <- kmeans.resultado$cluster
base
dplyr::count(base,cl_kmens)


In [ ]:

kmeans.resultado$cluster # el cluster asignado a cada observación de base.
kmeans.resultado$centers # Que representan estos centroides?


In [ ]:

# Se contrasta la clasificacion real con la definida despues de la clusterizacion
table(iris$Species, kmeans.resultado$cluster)
plot(base[c("Sepal.Length", "Sepal.Width")], col = kmeans.resultado$cluster)
# Se grafica los centros de los clusters 
points(kmeans.resultado$centers[,c("Sepal.Length", "Sepal.Width")], col = 1:3, pch = 8, cex=2)




In [ ]:
# Con ggplot 
# Agregar columna a iris con los clusters
# guardar en un **data.frame** los centros y agregar columna con colores 

In [ ]:

# Se calcula la distancia entre cada observación y los centroides de los clusters

centroides <- kmeans.resultado$centers[kmeans.resultado$cluster, ]
head(centroides)
distancias <- sqrt(rowSums((base - centroides)^2))
distancias

In [ ]:

# Se ordenan los registros con mayor distancia a sus respectivos centroides y se toman los top 5

outliers <- order(distancias, decreasing=T)[1:5]

# Cuales son los outliers

print(outliers)
print(base[outliers,])



In [ ]:
# Se grafican los clusters
plot(base[,c("Sepal.Length", "Sepal.Width")], pch="o", col=kmeans.resultado$cluster, cex=0.3)
#Se marcan los centroides de los clusters
points(kmeans.resultado$centers[,c("Sepal.Length", "Sepal.Width")], col=1:3, pch=8, cex=1.5)




In [ ]:
# Se grafican los outliers
plot(base[,c("Sepal.Length", "Sepal.Width")], pch="o", col=kmeans.resultado$cluster, cex=0.3)
points(kmeans.resultado$centers[,c("Sepal.Length", "Sepal.Width")], col=1:3, pch=8, cex=1.5)
points(base[outliers, c("Sepal.Length", "Sepal.Width")], pch="+", col=4, cex=1.5)


 ## Número de clusters

 Una de las opciones más utilizadas es la regla del "codo". Otra manera es estudiando el ancho de "silueta" de cada observación.

In [ ]:
names(base)
base$cl_kmens <- NULL

In [ ]:
names(base)

In [ ]:
# Generar un ciclo for que implemente el algoritmo kmeans con centros 2:50
# En cada iteración rescatar el total within sum squares (twss) y guardar dicho resultado en un dataframe
centros <- 2:50
twss_df <- data.frame()









In [ ]:
# Graficar


## **NOTA:** Escala de datos 
En ocasiones, cuando cada variable (columna) de nuestro set de datos tienen diferentes escala de datos (por ejemplo una columna con valores entre 0 y 1 , mientras que otra columna está en escala de 10 a 1000, ciertos algoritmos son suceptibles a dichas escalas, por tanto se recomienda escalar nuestra matriz de datos.

Cuando nos referimos a escalar, es equivalente a generar columnas con igual media y varianza, usualmente media 0 y desv. estándar 1. Esto lo podemos lograr con la función `scale()`

# Cluster jerárquico


Para ajustar un cluster jerárquico, en R contamos con la función `hclust()`

In [ ]:
library(ggplot2)
## Clústers  jerárquicos
muestraIris <- iris
muestraIris$Species <- NULL


## Cluster aglomerativo
# Se genera el cluster aglomerativo con linkage completo.

hc <- hclust(dist(muestraIris), method="average")


In [ ]:

plot(hc, 
     hang   = -1, #posición inicio dendograma
     labels = iris$Species,
     cex    = .6)



In [ ]:

# Finalmente  se corta el dendograma en K clusters
K <- 5

plot(hc, hang = -1, labels=iris$Species)
rect.hclust(hc, k=K)


In [ ]:
groups <- cutree(hc, k=K)

plot(iris[,c("Sepal.Length", "Sepal.Width")], col = groups)



In [ ]:

# gráfico de siluetas  ----------------------------------------------------
plot(cluster::silhouette(groups,dist(muestraIris))) 

sil <- cluster::silhouette(groups,dist(muestraIris))


str(sil)
mean(sil[,3])

## DBSCAN

In [ ]:
#install.packages("dbscan")
#install.packages("mlbench")

In [ ]:
library(dbscan)
library(mlbench)

In [ ]:
# iris --------------------------------------------------------------------
data(iris)
iris <- as.matrix(iris[,1:4])

## find suitable eps parameter using a k-NN plot for k = dim + 1
## Look for the knee!

minPts <- 4+1

kNNdistplot(iris, k = minPts)
abline(h=.6, col = "red", lty=2)



In [ ]:
###### Es lo mismo que 
sort(kNNdist(iris, k = minPts)) -> distancias_knn

plot(distancias_knn,type = "l")
abline( h =.6, col = "red", lty=2)
######


In [ ]:
res <- dbscan(iris, eps = .4, minPts = minPts)
res

pairs(iris, col = res$cluster + 1L)



In [ ]:
# espiral -----------------------------------------------------------------
set.seed(666)
spiral <- mlbench.spirals(6000, 1, sd = 0.06)
spiral <- cbind(spiral$x, label = spiral$classes)
colnames(spiral)[c(1, 2)] <- c("x1", "x2")
spiral <- as.data.frame(spiral)
spiral$label <- factor(spiral$label)



ggplot(spiral) + 
  aes(x = x1, y = x2) + 
  geom_point()


In [ ]:
minPts <- 6

kNNdistplot(spiral[,1:2], k = minPts)
#abline(h=.04, col = "red", lty = 2)



In [ ]:
eps <- 0.04

res_spiral <- dbscan(spiral[,1:2], eps = eps, minPts = minPts)
res_spiral



In [ ]:
res_spiral$cluster

spiral2 <- spiral 

spiral2$cl <- res_spiral$cluster #== 0L



In [ ]:

ggplot(spiral2) + 
  aes(x = x1, y = x2, color = as.factor(cl) ) + 
  geom_point()

## OPTICS

In [ ]:
set.seed(2)
n <- 400

x <- cbind(
  x = runif(4, 0, 1) + rnorm(n, sd=0.1),
  y = runif(4, 0, 1) + rnorm(n, sd=0.1)
)

plot(x, col=rep(1:4, time = 100))


In [ ]:

### run OPTICS (Note: we use the default eps calculation)
res <- optics(x, minPts = 5)
res




In [ ]:
### get order
res$order

### plot produces a reachability plot
plot(res)
abline(h = .05, col = "red", lty = 2)




In [ ]:

### extract a DBSCAN clustering by cutting the reachability plot at eps_cl
res2 <- extractDBSCAN(res, eps_cl = .054)
res2



In [ ]:
plot(res2)  ## black is noise


In [ ]:
hullplot(x, res2)


